## Get Best-Performing Hyper Parameters

In [202]:
client = pymongo.MongoClient(serverSelectionTimeoutMS=100, host="127.0.0.1")
db = client.metrics_causal_model_reranker

In [216]:
import pandas as pd
import pymongo

def get_df(collection, db):
    assert collection.endswith("_VD")
    cols = ["f1_score", "precision", "recall", "asof", "best_top_n", "C", "best_max_parses", 
        #"best_max_upd", 
        "max_update_items", 
        # "best_min_prob", "extractors", 
        "initial_weight", "loss_type",\
        #"min_feat_freq",
        "pa_type"]
    if "INSENS" in collection:
        cols = [c for c in cols if c != "loss_type"]

    elif "PCPTRN" in collection:
        cols = ["f1_score", "precision", "recall", "asof", "best_top_n", "learning_rate", "best_max_parses", 
        #"best_max_upd", 
        "max_update_items", 
        # "best_min_prob", "extractors", 
        "initial_weight"]
        #"min_feat_freq"]
    
    project = {
            "params": "$parameters",
            "micro_f1": "$MICRO_F1",
            "asof": "$asof",
            "_id": 1
        }
    feats_pipeline = [{ "$project": project }]
    rows = [row for row in db[collection].aggregate(feats_pipeline)]

    results = []
    for r in rows:
        d = dict(r["params"])
        d.update(r["micro_f1"])
        d["asof"] = str(r["asof"])
        results.append(d)
    
    df = pd.DataFrame(results)
    df = df.sort_values(by="f1_score",ascending=False)
    return df[cols]

In [217]:
# MONGO_COLLECTION = "SC_RE-RANKER_HYPER_PARAM_VD"
# MONGO_COLLECTION = "SC_COST_INSENS_RE-RANKER_HYPER_PARAM_VD"

In [218]:
df_pa = get_df("CB_RE-RANKER_HYPER_PARAM_VD", db)
df_pa.head(1)

,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,initial_weight,loss_type,pa_type
37,0.746014,0.777294,0.717153,2019-06-18 01:55:37.752000,2,0.0025,300,1,0.01,ml,1


In [219]:
df_pa = get_df("CB_COST_INSENS_RE-RANKER_HYPER_PARAM_VD", db)
df_pa.head(1)

,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,initial_weight,pa_type
30,0.737302,0.807826,0.678102,2019-06-23 15:33:17.018000,1,0.1,300,1,0.01,1


In [220]:
df_pa = get_df("CB_STR_PCPTRN_RE-RANKER_HYPER_PARAM_VD", db)
df_pa.head(1)

,f1_score,precision,recall,asof,best_top_n,learning_rate,best_max_parses,max_update_items,initial_weight
4,0.74097,0.787033,0.7,2019-06-24 19:41:12.771000,2,0.05,300,1,0.01
